# -- DAY 9 --


In [2]:
# --- Part One ---

#from itertools import combinations
from icecream import ic

test = "2333133121414131402"

retvalue = 0

class Disk:
    def __init__(self, diskmap = None):
        self.clusters = {}
        if diskmap: self.import_diskmap(diskmap)
    
    def size(self):
        return len(self.clusters)
    
    def info(self):
        info = {}
        info["size"] = self.size()
        info["free"] = len([k for k,v in self.clusters.items() if v == None])
        info["used"] = self.size() - info["free"]
        return info

    def import_diskmap(self, diskmap):
        """
        Import a disk map
        The disk map uses a dense format to represent the layout of files and free space on the disk.
        The digits alternate between indicating the length of a file and the length of free space.
        """
        dm = map(int,list(diskmap))
        cl = 0
        id = 0
        is_file = True
        for c in dm:
            for i in range(c):
                if is_file: self.clusters[cl] = id
                else: self.clusters[cl] = None
                cl += 1
            is_file = not is_file
            id = id + 1 if is_file else id
    
    def defrag(self):
        free = [k for k,v in self.clusters.items() if v == None]
        used = list(reversed([(k, v) for k,v in self.clusters.items() if v != None]))        
        while used[0][0] > free[0]: # if used to move is in a cluster following the first free cluster
            from_cluster = used.pop(0)
            to_cluster = free.pop(0)
            self.clusters[to_cluster] = from_cluster[1]
            self.clusters[from_cluster[0]] = None
        return

    def checksum(self):
        return sum([k * v for k,v in self.clusters.items() if v != None],0)


with open("input.txt","r") as file:
    #file = test.splitlines() # decomment to use test 
    diskmap = [l for l in file][0].strip()

    disk = Disk(diskmap)

    print(disk.info())
    disk.defrag()

    #m = "".join([str(c) if c!=None else '.' for c in disk.clusters.values()])
    # print (m)

    retvalue = disk.checksum()

print (f"The solution of Puzzle 1 is: {retvalue}")


{'size': 94798, 'free': 44967, 'used': 49831}
The solution of Puzzle 1 is: 6288599492129


In [4]:
# --- Part Two ---

# This solution uses objects defined in part 1

test = "2333133121414131402"

retvalue = 0

# --- class patch ---
# Modify the original Disk class
# `defrag` method now accepts the "method" parameter which can be either "compact" or "files"
# If method is "compact", the method will compact the disk using original `defrag` method
# If method is "files", the method will compact the disk mantaining the file unfragmented
_defrag_compact = Disk.defrag

def _defrag_files(self):
    # map files and free clusters in clusters
    files = {} # {id: [start, len]}
    frees = {} # {id: [start, len]}
    last = -1
    id_free = -1
    for k,v in self.clusters.items():
        if v == None:
            if last != None:
                id_free = k
            frees.setdefault(id_free, [id_free, 0])[1] += 1
        else:
            files.setdefault(v, [k, 0])[1] += 1
        last = v

    # DEBUG print("".join([str(c) if c!=None else '.' for c in self.clusters.values()]))

    for file_id, (file_cluster, file_size) in reversed(list(files.items())):
        for free_id, (free_cluster, free_size) in frees.items():
            if free_size >= file_size and file_cluster > free_cluster:
                self._move_file(file_cluster, free_cluster, file_size)

                frees[free_id][0] += file_size
                frees[free_id][1] -= file_size
                # DEBUG print("".join([str(c) if c!=None else '.' for c in self.clusters.values()]))
                break
            

def _move_file(self, from_cluster, to_cluster, size):
    # check if to_cluster is free until size is reached
    for i in range(size):
        if self.clusters[to_cluster + i] != None:
            raise ValueError(f"Cluster {to_cluster + i} is not free")
    # move file
    for i in range(size):
        self.clusters[to_cluster + i] = self.clusters[from_cluster + i]
        self.clusters[from_cluster + i] = None

def new_defrag(self, method = "compact"):
    if method == "compact":
        _defrag_compact(self)
    elif method == "files":
        _defrag_files(self)
    else:
        raise ValueError(f"Invalid method: {method}")

Disk.defrag = new_defrag
Disk._move_file = _move_file

# --- end of class patch ---

with open("input.txt","r") as file:
    # file = test.splitlines() # decomment to use test 
    diskmap = [l for l in file][0].strip()

    disk = Disk(diskmap)

    print(disk.info())
    disk.defrag("files")

    retvalue = disk.checksum()
print (f"The solution of Puzzle 2 is: {retvalue}")



{'size': 94798, 'free': 44967, 'used': 49831}
The solution of Puzzle 2 is: 6321896265143
